In [45]:
import datetime

from numba import jit
from numba import int32

import numpy as np
import xarray as xr

In [66]:
def solve_rotunno_case_two(xin=161,zetaN=81,tauN=32,kN=21,
                           xi0=0.2, latitude=5, h=1500, 
                           delTheta=6, theta0=300, theta1=350, 
                           tropHeight=12000):
    
    # Time interval
    delTau = 2*np.pi/tauN

    # Initialise solution arrays
    psi = np.zeros((xiN,zetaN,tauN)) 
    u = np.zeros((xiN,zetaN,tauN))
    v = np.zeros((xiN,zetaN,tauN))
    w = np.zeros((xiN,zetaN,tauN))
    b = np.zeros((xiN,zetaN,tauN))

    # Initialise domains
    xi = np.linspace(-2,2,xiN)
    zeta = np.linspace(0,4,zetaN)
    tau = np.arange(0,2*np.pi,delTau)
    k = np.linspace(0,20,kN)

    # Define constants
    omega = 7.2921159 * 10.0 ** (-5)
    g = 9.80665

    # Define parameters
    xi0 = 0.2
    latitude = 5.0
    h = 1500
    delTheta = 6.0 # Daily change in potential temperature
    theta0 = 300.0
    theta1 = 350.0
    tropHeight = 12000.0

    # Calculate nondimensional model parameters from inputs
    f = 2.0*omega*np.sin(latitude * np.pi / 180.0)
    N = np.sqrt((g/theta0)*(theta1-theta0)/tropHeight) # Brunt Vaisala Frequency
    beta = omega**2/(N*np.sqrt(omega**2-f**2))
    Atilde = .5*delTheta*(g/(np.pi*theta0))*h**(-1)*omega**(-3)/(12*60*60)
    
    psi, u, w = integrate_case_two(xi,zeta,tau,k,xi0,beta,Atilde)
    
    ds = xr.Dataset({'psi':(('tau','zeta','xi'),psi), 
                     'u':(('tau','zeta','xi'),u), 
                     'w':(('tau','zeta','xi'),w)},
                    {'tau': tau, 'zeta':zeta, 'xi':xi},
                    {'xi0':xi0, 'latitude':latitude, 'h':h, 
                     'delTheta':delTheta, 'theta0':theta0,
                     'theta1':theta1, 'tropHeight':tropHeight})
    
    now = str(datetime.datetime.now())[0:-7]
    now=now.replace('-', '').replace(':', '').replace(' ', '_')
    ds.to_netcdf('../datasets/rotunno_case_2_{}.nc'.format(now))

In [67]:
@jit()
def integrate_case_two(xi,zeta,tau,k,xi0,beta,Atilde):
    psi = np.zeros((tau.size, zeta.size, xi.size))
    u = np.zeros((tau.size, zeta.size, xi.size))
    w = np.zeros((tau.size, zeta.size, xi.size))
    psi_integrand = np.zeros(k.size)
    u_integrand = np.zeros(k.size)
    w_integrand = np.zeros(k.size)
    # Perform numerical integration
    for i in range(xi.size):
        for j in range(zeta.size):
            for l in range(tau.size):
                for m in range(k.size):
                    psi_integrand[m] = (np.cos(k[m]*xi[i])
                                        *np.exp(-xi0*k[m])
                                        *(np.sin(k[m]*zeta[j]+tau[l])
                                          -np.exp(-zeta[j])*np.sin(tau[l]))
                                        /(1+k[m]**2))
                    u_integrand[m] = (k[m]*np.sin(k[m]*xi[i])
                                      *np.exp(-xi0*k[m])
                                      *(np.sin(k[m]*zeta[j]+tau[l])
                                        -np.exp(-zeta[j])*np.sin(tau[l]))
                                      /(1+k[m]**2))
                    w_integrand[m] = (np.cos(k[m]*xi[i])
                                      *np.exp(-xi0*k[m])
                                      *(k[m]*np.cos(k[m]*zeta[j]+tau[l])
                                        +np.exp(-zeta[j])*np.sin(tau[l]))
                                      /(1+k[m]**2))
                    
                psi[l,j,i] = np.trapz(k,psi_integrand)
                u[l,j,i] = np.trapz(k,u_integrand)
                w[l,j,i] = np.trapz(k,w_integrand)
    # Scale
    psi = -beta * Atilde * psi
    u = -beta * Atilde * u
    w = -beta * Atilde * w
    return psi, u, w